<a href="https://colab.research.google.com/github/objectc/CNN-with-TensorFlow2.0-and-Keras/blob/master/tansfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import all the necessary files!
import tensorflow as tf
from tensorflow import keras
import sklearn

In [0]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [4]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


--2019-05-15 01:55:35--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.128, 2607:f8b0:4001:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   147MB/s    in 0.6s    

2019-05-15 01:55:36 (147 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
local_weight_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (96, 96, 3), include_top = False, weights = None)


In [0]:
pre_trained_model.load_weights(local_weight_file)
for layer in pre_trained_model.layers:
  layer.trainable = False

In [7]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 47, 47, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 47, 47, 32)   0           batch_normalization_v1[0][0]     
_______________________________________________________________________________________

In [0]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [0]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0
sklearn.utils.shuffle(X_train_full, y_train_full)
X_train = X_train_full[0:-5000]
y_train = y_train_full[0:-5000]
X_valid = X_train_full[-5000:]
y_valid = y_train_full[-5000:]

In [0]:
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [12]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
immediateModel = keras.Model(pre_trained_model.inputs, last_layer.output)
# last_layer_fuc = keras.backend.function(pre_trained_model.inputs, last_layer.output)

last layer output shape:  (None, 4, 4, 768)


In [0]:
keras.backend.clear_session()
train_input = keras.layers.Input((32, 32, 3))
x = keras.layers.Lambda(lambda image: tf.image.resize(image, (96, 96)))(train_input)
x = immediateModel(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
# x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(len(classes), activation=keras.activations.softmax)(x)

In [0]:
model = keras.Model(train_input, x)

In [0]:
model.compile(optimizer=keras.optimizers.Adam(decay=0.0001), 
              loss=keras.losses.sparse_categorical_crossentropy, 
              metrics=[keras.metrics.sparse_categorical_accuracy])

In [0]:
from keras_preprocessing.image import ImageDataGenerator

In [0]:
traingen = ImageDataGenerator(
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
X_train_gen = traingen.flow(X_train, y_train)

In [18]:
history = model.fit(X_train_gen, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
1407/1407 [==============================] - 50s 36ms/step - loss: 0.9512 - sparse_categorical_accuracy: 0.6742 - val_loss: 0.6890 - val_sparse_categorical_accuracy: 0.7642
Epoch 2/100
1407/1407 [==============================] - 44s 32ms/step - loss: 0.7644 - sparse_categorical_accuracy: 0.7332 - val_loss: 0.6348 - val_sparse_categorical_accuracy: 0.7866
Epoch 3/100
1407/1407 [==============================] - 43s 30ms/step - loss: 0.6907 - sparse_categorical_accuracy: 0.7586 - val_loss: 0.6037 - val_sparse_categorical_accuracy: 0.7952
Epoch 4/100
1407/1407 [==============================] - 45s 32ms/step - loss: 0.6494 - sparse_categorical_accuracy: 0.7748 - val_loss: 0.5956 - val_sparse_categorical_accuracy: 0.7928
Epoch 5/100
1407/1407 [==============================] - 43s 31ms/step - loss: 0.6123 - sparse_categorical_accuracy: 0.7869 - val_loss: 0.5591 - val_sparse_categorical_accuracy: 0.8078
Epoch 6/100
1407/1407 [==============================] - 44s 31ms/step - lo

KeyboardInterrupt: ignored